<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Alt1:-Layers-by-hand" data-toc-modified-id="Alt1:-Layers-by-hand-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Alt1: Layers by hand</a></span></li><li><span><a href="#Alt2:-Layers-via-tf.layers.dense" data-toc-modified-id="Alt2:-Layers-via-tf.layers.dense-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Alt2: Layers via tf.layers.dense</a></span></li></ul></div>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# Setup vars for the MINST data set
inputs = 28 * 28    # image dim in pixels
hidden1 = 300
hidden2 = 100
outputs = 10

lr = 0.01
epochs = 40
batchSize = 50

mninst = input_data.read_data_sets("/tmp/data/")

# Alt1: Layers by hand

In [5]:
# Build functions to create NN layer

def nnLayer(x, neurons, name, activation = None):
    with tf.name_scope(name):
        # Get number of features
        inputs = int(x.shape[1])
        
        # Truncated normal (Gaussian) distribution with a standard deviation of -> 2/sqrt(inputs)
        # It is important to initialize connection weights randomly for all hidden layers to avoid 
        # any symmetries that the Gradient Descent algorithm would be unable to break.
        stddev = 2 / np.sqrt(inputs)
        
        # init the weights
        # tf.truncate_normal:
        #    The generated values follow a normal distribution with specified mean and standard deviation, 
        #    except that values whose magnitude is more than 2 standard deviations from the mean are dropped and re-picked.
        init = tf.truncated_normal( (inputs, neurons), stddev = stddev)
        
        w = tf.Variable(init, name = 'kernel')
        b = tf.Variable(tf.zeros([neurons]), name = 'bias')
        z = tf.matmul(x, w) + b
        
        if activation is not None:
            return activation(z)
        else:
            return z

In [9]:
# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

with tf.name_scope("neuralNet"):
    layer1 = nnLayer(x, hidden1, "hLayerOne", tf.nn.relu)
    layer2 = nnLayer(layer1, hidden2, "hLayerTwo", tf.nn.relu)
    yH = nnLayer(layer2, outputs, "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

# Save our trained model parameters to disk
saver = tf.train.Saver()

In [12]:
# Execute the TF CG
with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            sess.run([opt], feed_dict = {x: xBatch, y: yBatch})
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    savePath = saver.save(sess, "./DNN-Using-Plain-TensorFlow-Model.ckpt")

0 Train Acc:  0.98 Test Acc:  0.9174
1 Train Acc:  0.98 Test Acc:  0.9336
2 Train Acc:  0.94 Test Acc:  0.9407
3 Train Acc:  0.96 Test Acc:  0.9443
4 Train Acc:  0.94 Test Acc:  0.9489
5 Train Acc:  0.98 Test Acc:  0.9521
6 Train Acc:  0.96 Test Acc:  0.9571
7 Train Acc:  0.98 Test Acc:  0.9604
8 Train Acc:  0.98 Test Acc:  0.9624
9 Train Acc:  0.96 Test Acc:  0.9621
10 Train Acc:  0.92 Test Acc:  0.9664
11 Train Acc:  0.92 Test Acc:  0.9661
12 Train Acc:  0.98 Test Acc:  0.9669
13 Train Acc:  1.0 Test Acc:  0.9693
14 Train Acc:  0.94 Test Acc:  0.9713
15 Train Acc:  1.0 Test Acc:  0.9712
16 Train Acc:  0.98 Test Acc:  0.9718
17 Train Acc:  1.0 Test Acc:  0.9724
18 Train Acc:  0.98 Test Acc:  0.9726
19 Train Acc:  1.0 Test Acc:  0.9723
20 Train Acc:  1.0 Test Acc:  0.9741
21 Train Acc:  0.98 Test Acc:  0.9742
22 Train Acc:  0.98 Test Acc:  0.9754
23 Train Acc:  0.96 Test Acc:  0.9741
24 Train Acc:  0.98 Test Acc:  0.9754
25 Train Acc:  1.0 Test Acc:  0.9756
26 Train Acc:  1.0 Test Acc:

In [28]:
# Use the saved model
# We don't have new iamges, so we'll just reuse the test images

xNew = mninst.test.images
yNew = mninst.test.labels
    
with tf.Session() as sess:
    saver.restore(sess, "./DNN-Using-Plain-TensorFlow-Model.ckpt")  
    newZ = yH.eval(feed_dict = {x: xNew})
    yHNew = np.argmax(newZ, axis = 1)
    
print("New data acc: %.2f%%" % (np.sum(yHNew == yNew) / len(yNew) * 100))

INFO:tensorflow:Restoring parameters from ./DNN-Using-Plain-TensorFlow-Model.ckpt
New data acc: 97.91%


# Alt2: Layers via tf.layers.dense

In [35]:
# Reset the TF CG
tf.reset_default_graph()

In [36]:
# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("neuralNetEasy"):
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

# Save our trained model parameters to disk
saver = tf.train.Saver()

In [37]:
# Execute the TF CG
with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            sess.run([opt], feed_dict = {x: xBatch, y: yBatch})
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)
    
    savePath = saver.save(sess, "./DNN-Using-Plain-TensorFlow-Model-Easy.ckpt")

0 Train Acc:  0.94 Test Acc:  0.9055
10 Train Acc:  0.96 Test Acc:  0.9612
20 Train Acc:  0.96 Test Acc:  0.971
30 Train Acc:  1.0 Test Acc:  0.9755
FINAL ::  Train Acc:  0.96 Test Acc:  0.9775
